**Introduction**

This model predicts whether or not a child has pneumonia and if so, is it bacterial or viral pneumonia. This kernel is part of an end to end ml solution - starting with a model and ending with a live web app. A user is able to submit a chest x-ray and get an instant prediction. The app was created using Tensorflow.js.

 All javascript, html and css files needed to construct the app are available on github.<br>

Web App: http://child.test.woza.work/<br>
Github: https://github.com/vbookshelf/Child-Pneumonia-Diagnoser<br>

This Kaggle dataset was set up as a binary classification problem - pneumonia or no pneumonia. I found that the images had file names containing normal, bacteria or virus so I extracted these and used them as target classes to create a 3 class classification problem - Normal, Viral, Bacterial.

This solution uses transfer learning applied to a MobileNet model. All training was done in this kernel. MobileNet was designed for the web. It is small (aprox. 35MB) and runs fast. 

**Findings**<br>

From the resulting confusion matrix and classification report it appears that the model is very good at predicting whether or not pneumonia is present, however it also seems to be more likely to predict bacterial pneumonia than viral pneumonia. Initially I was going to try to reduce this sensitivity to bacterial pneumonia but I decided not to do this. Because bacterial pneumonia is more serious than viral pneumonia I left the model as is.

One piece of information that would be good to have is an estimate of human level performance on this task. An F1 score of a Radiologist's ability on this dataset would allow a realistic assessment to be made of this model's performance. The developers of [CheXNet](https://stanfordmlgroup.github.io/projects/chexnet/) used this approach to determine their model's performance. They computed the average F1 score of 4 Radiologists and compared it to CheXNet's F1 score.


In [ ]:
from numpy.random import seed
seed(101)
from tensorflow import set_random_seed
set_random_seed(101)

import pandas as pd
import numpy as np
import keras
from keras import backend as K
from keras.layers.core import Dense, Dropout
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.metrics import categorical_accuracy, top_k_categorical_accuracy

import os
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import itertools
import shutil
import matplotlib.pyplot as plt
%matplotlib inline


### Check how many files in each folder
Note: In some folders there is a non image file called  '.DS_Store'

In [ ]:
# Train
# normal
print(len(os.listdir('../input/chest_xray/chest_xray/train/NORMAL')))
# pneumonia
print(len(os.listdir('../input/chest_xray/chest_xray/train/PNEUMONIA')))


In [ ]:
# Val
# normal
print(len(os.listdir('../input/chest_xray/chest_xray/val/NORMAL')))
# pneumonia
print(len(os.listdir('../input/chest_xray/chest_xray/val/PNEUMONIA')))

In [ ]:
# Test
# normal
print(len(os.listdir('../input/chest_xray/chest_xray/test/NORMAL')))
# pneumonia
print(len(os.listdir('../input/chest_xray/chest_xray/test/PNEUMONIA')))

In [ ]:
os.listdir('../input/chest_xray/chest_xray/test')

### Create a Dataframe containing all images

In [ ]:
# create a list of files in each folder
train_normal_list = os.listdir('../input/chest_xray/chest_xray/train/NORMAL')
train_pneu_list = os.listdir('../input/chest_xray/chest_xray/train/PNEUMONIA')
val_normal_list = os.listdir('../input/chest_xray/chest_xray/val/NORMAL')
val_pneu_list = os.listdir('../input/chest_xray/chest_xray/val/PNEUMONIA')
test_normal_list = os.listdir('../input/chest_xray/chest_xray/test/NORMAL')
test_pneu_list = os.listdir('../input/chest_xray/chest_xray/test/PNEUMONIA')

In [ ]:
def assign_pneu_type(x):
    x = str(x)
    if 'virus' in x:
        return 'viral'
    if 'bacteria' in x:
        return 'bacterial'

In [ ]:
# TRAIN_NORMAL
# create the dataframe
df_train_normal = pd.DataFrame(train_normal_list, columns=['image_id'])
# delete the entry named .DS_Store
df_train_normal = df_train_normal[df_train_normal['image_id'] != '.DS_Store']
# create a new target column
df_train_normal['target'] = 'normal'

# TRAIN_PNEU
# create the dataframe
df_train_pneu = pd.DataFrame(train_pneu_list, columns=['image_id'])
# delete the entry named .DS_Store
df_train_pneu = df_train_pneu[df_train_pneu['image_id'] != '.DS_Store']
# create a target column that's a copy of the image column
df_train_pneu['target'] = df_train_pneu['image_id']
# apply the function to this target column
df_train_pneu['target'] = df_train_pneu['target'].apply(assign_pneu_type)

# VAL_NORMAL
# create the dataframe
df_val_normal = pd.DataFrame(val_normal_list, columns=['image_id'])
# delete the entry named .DS_Store
df_val_normal = df_val_normal[df_val_normal['image_id'] != '.DS_Store']
# create a new target column
df_val_normal['target'] = 'normal'


# VAL_PNEU
# create the dataframe
df_val_pneu = pd.DataFrame(val_pneu_list, columns=['image_id'])
# delete the entry named .DS_Store
df_val_pneu = df_val_pneu[df_val_pneu['image_id'] != '.DS_Store']
# create a target column that's a copy of the image column
df_val_pneu['target'] = df_val_pneu['image_id']
# apply the function to this target column
df_val_pneu['target'] = df_val_pneu['target'].apply(assign_pneu_type)


# TEST_NORMAL
# create the dataframe
df_test_normal = pd.DataFrame(test_normal_list, columns=['image_id'])
# delete the entry named .DS_Store
df_test_normal = df_test_normal[df_test_normal['image_id'] != '.DS_Store']
# create a new target column
df_test_normal['target'] = 'normal'


# TEST_PNEU
# create the dataframe
df_test_pneu = pd.DataFrame(test_pneu_list, columns=['image_id'])
# delete the entry named .DS_Store
df_test_pneu = df_test_pneu[df_test_pneu['image_id'] != '.DS_Store']
# create a target column that's a copy of the image column
df_test_pneu['target'] = df_test_pneu['image_id']
# apply the function to this target column
df_test_pneu['target'] = df_test_pneu['target'].apply(assign_pneu_type)


In [ ]:
# Concat the dataframes
df_data = \
pd.concat([df_train_normal,df_train_pneu,df_val_normal,df_val_pneu,df_test_normal,
           df_test_pneu],axis=0).reset_index(drop=True)

# shuffle
df_data = shuffle(df_data)

df_data.shape

In [ ]:
# Check the target distribution
df_data['target'].value_counts()

In [ ]:
df_data.head()

### Create the directory structure
The original dataset directory structure was set up to support a binary classification problem. Because we will be predicting 3 classes we need to change this structure. In these folders we will store the images that will later be fed to the Keras generators.

In [ ]:
# Create a new directory
base_dir = 'base_dir'
os.mkdir(base_dir)


#[CREATE FOLDERS INSIDE THE BASE DIRECTORY]

# now we create 3 folders inside 'base_dir':

# train
    # normal
    # bacterial
    # viral

# val
    # normal
    # bacterial
    # viral


# create a path to 'base_dir' to which we will join the names of the new folders
# train_dir
train_dir = os.path.join(base_dir, 'train_dir')
os.mkdir(train_dir)

# val_dir
val_dir = os.path.join(base_dir, 'val_dir')
os.mkdir(val_dir)


# [CREATE FOLDERS INSIDE THE TRAIN AND VALIDATION FOLDERS]
# Inside each folder we create seperate folders for each class

# create new folders inside train_dir
normal = os.path.join(train_dir, 'normal')
os.mkdir(normal)
bacterial = os.path.join(train_dir, 'bacterial')
os.mkdir(bacterial)
viral = os.path.join(train_dir, 'viral')
os.mkdir(viral)



# create new folders inside val_dir
normal = os.path.join(val_dir, 'normal')
os.mkdir(normal)
bacterial = os.path.join(val_dir, 'bacterial')
os.mkdir(bacterial)
viral = os.path.join(val_dir, 'viral')
os.mkdir(viral)


In [ ]:
os.listdir('base_dir/train_dir')

### Create Train and Val Sets

In [ ]:
y = df_data['target']

df_train, df_val = train_test_split(df_data, test_size=0.15, random_state=101, stratify=y)

print(df_train.shape)
print(df_val.shape)

In [ ]:
# check df_train class distribution
df_train['target'].value_counts()

In [ ]:
# check df_val class distribution
df_val['target'].value_counts()

### Transfer the Images into the Folders

In [ ]:
df_data.head()

In [ ]:
# Set the image_id as the index in df_data
df_data.set_index('image_id', inplace=True)

In [ ]:
# Get a list of images in each of the folders
train_normal_list = os.listdir('../input/chest_xray/chest_xray/train/NORMAL')
train_pneu_list = os.listdir('../input/chest_xray/chest_xray/train/PNEUMONIA')
val_normal_list = os.listdir('../input/chest_xray/chest_xray/val/NORMAL')
val_pneu_list = os.listdir('../input/chest_xray/chest_xray/val/PNEUMONIA')
test_normal_list = os.listdir('../input/chest_xray/chest_xray/test/NORMAL')
test_pneu_list = os.listdir('../input/chest_xray/chest_xray/test/PNEUMONIA')

# Get a list of train and val images
train_list = list(df_train['image_id'])
val_list = list(df_val['image_id'])

# Transfer the train images

for image in train_list:
    
    fname = image
    label = df_data.loc[image,'target']
    
    if fname in train_normal_list:
        # source path to image
        src = os.path.join('../input/chest_xray/chest_xray/train/NORMAL', fname)
        # destination path to image
        dst = os.path.join(train_dir, label, fname)
        # copy the image from the source to the destination
        shutil.copyfile(src, dst)

    if fname in train_pneu_list:
        # source path to image
        src = os.path.join('../input/chest_xray/chest_xray/train/PNEUMONIA', fname)
        # destination path to image
        dst = os.path.join(train_dir, label, fname)
        # copy the image from the source to the destination
        shutil.copyfile(src, dst)
    
    if fname in val_normal_list:
        # source path to image
        src = os.path.join('../input/chest_xray/chest_xray/val/NORMAL', fname)
        # destination path to image
        dst = os.path.join(train_dir, label, fname)
        # copy the image from the source to the destination
        shutil.copyfile(src, dst)
    
    if fname in val_pneu_list:
        # source path to image
        src = os.path.join('../input/chest_xray/chest_xray/val/PNEUMONIA', fname)
        # destination path to image
        dst = os.path.join(train_dir, label, fname)
        # copy the image from the source to the destination
        shutil.copyfile(src, dst)
        
    if fname in test_normal_list:
        # source path to image
        src = os.path.join('../input/chest_xray/chest_xray/test/NORMAL', fname)
        # destination path to image
        dst = os.path.join(train_dir, label, fname)
        # copy the image from the source to the destination
        shutil.copyfile(src, dst)
    
    if fname in test_pneu_list:
        # source path to image
        src = os.path.join('../input/chest_xray/chest_xray/test/PNEUMONIA', fname)
        # destination path to image
        dst = os.path.join(train_dir, label, fname)
        # copy the image from the source to the destination
        shutil.copyfile(src, dst)
        
        


# Transfer the val images

for image in val_list:
    
    fname = image
    label = df_data.loc[image,'target']
    
    if fname in train_normal_list:
        # source path to image
        src = os.path.join('../input/chest_xray/chest_xray/train/NORMAL', fname)
        # destination path to image
        dst = os.path.join(val_dir, label, fname)
        # copy the image from the source to the destination
        shutil.copyfile(src, dst)

    if fname in train_pneu_list:
        # source path to image
        src = os.path.join('../input/chest_xray/chest_xray/train/PNEUMONIA', fname)
        # destination path to image
        dst = os.path.join(val_dir, label, fname)
        # copy the image from the source to the destination
        shutil.copyfile(src, dst)
    
    if fname in val_normal_list:
        # source path to image
        src = os.path.join('../input/chest_xray/chest_xray/val/NORMAL', fname)
        # destination path to image
        dst = os.path.join(val_dir, label, fname)
        # copy the image from the source to the destination
        shutil.copyfile(src, dst)
    
    if fname in val_pneu_list:
        # source path to image
        src = os.path.join('../input/chest_xray/chest_xray/val/PNEUMONIA', fname)
        # destination path to image
        dst = os.path.join(val_dir, label, fname)
        # copy the image from the source to the destination
        shutil.copyfile(src, dst)
    
    if fname in test_normal_list:
        # source path to image
        src = os.path.join('../input/chest_xray/chest_xray/test/NORMAL', fname)
        # destination path to image
        dst = os.path.join(val_dir, label, fname)
        # copy the image from the source to the destination
        shutil.copyfile(src, dst)
    
    if fname in test_pneu_list:
        # source path to image
        src = os.path.join('../input/chest_xray/chest_xray/test/PNEUMONIA', fname)
        # destination path to image
        dst = os.path.join(val_dir, label, fname)
        # copy the image from the source to the destination
        shutil.copyfile(src, dst)
        




In [ ]:
# check how many train images we have in each folder

print(len(os.listdir('base_dir/train_dir/normal')))
print(len(os.listdir('base_dir/train_dir/bacterial')))
print(len(os.listdir('base_dir/train_dir/viral')))

In [ ]:
# check how many val images we have in each folder

print(len(os.listdir('base_dir/val_dir/normal')))
print(len(os.listdir('base_dir/val_dir/bacterial')))
print(len(os.listdir('base_dir/val_dir/viral')))

### Copy the train images into aug_dir
aug_dir is where we temporarily store images from a given class before feeding them into the generator for augmentation.¶ 

We will not be augmenting on the fly. We will create augmented images, store them in folders together with the raw images and then feed these into the generators. I found that working this way makes the training process run faster. 

In [ ]:

class_list = ['normal','bacterial','viral']

for item in class_list:
    
    # We are creating temporary directories here because we delete these directories later
    # create a base dir
    aug_dir = 'aug_dir'
    os.mkdir(aug_dir)
    # create a dir within the base dir to store images of the same class
    img_dir = os.path.join(aug_dir, 'img_dir')
    os.mkdir(img_dir)

    # Choose a class
    img_class = item

    # list all images in that directory
    img_list = os.listdir('base_dir/train_dir/' + img_class)

    # Copy images from the class train dir to the img_dir e.g. class 'bacterial'
    for fname in img_list:
            # source path to image
            src = os.path.join('base_dir/train_dir/' + img_class, fname)
            # destination path to image
            dst = os.path.join(img_dir, fname)
            # copy the image from the source to the destination
            shutil.copyfile(src, dst)


    # point to a dir containing the images and not to the images themselves
    path = aug_dir
    save_path = 'base_dir/train_dir/' + img_class

    # Create a data generator
    datagen = ImageDataGenerator(
        rotation_range=10,
        width_shift_range=0.05,
        height_shift_range=0.05,
        zoom_range=0.05,
        horizontal_flip=True,
        fill_mode='nearest')

    batch_size = 50

    aug_datagen = datagen.flow_from_directory(path,
                                           save_to_dir=save_path,
                                           save_format='jpg',
                                                    target_size=(224,224),
                                                    batch_size=batch_size)
    
    
    # Generate the augmented images and add them to the training folders
    
    ###########
    
    num_aug_images_wanted = 5000 # total number of images we want to have in each class
    
    ###########
    
    num_files = len(os.listdir(img_dir))
    num_batches = int(np.ceil((num_aug_images_wanted-num_files)/batch_size))

    # run the generator and create augmented images
    for i in range(0,num_batches):

        imgs, labels = next(aug_datagen)
        
    # delete temporary directory with the raw image files
    shutil.rmtree('aug_dir')

In [ ]:
# Check how many train images we now have in each folder.
# This is the original images plus the augmented images.

print(len(os.listdir('base_dir/train_dir/normal')))
print(len(os.listdir('base_dir/train_dir/bacterial')))
print(len(os.listdir('base_dir/train_dir/viral')))

In [ ]:
# Check how many val images we have in each folder.

print(len(os.listdir('base_dir/val_dir/normal')))
print(len(os.listdir('base_dir/val_dir/bacterial')))
print(len(os.listdir('base_dir/val_dir/viral')))

### Visualize 50 augmented images

In [ ]:
# plots images with labels within jupyter notebook
# source: https://github.com/smileservices/keras_utils/blob/master/utils.py

def plots(ims, figsize=(12,6), rows=5, interp=False, titles=None): # 12,6
    if type(ims[0]) is np.ndarray:
        ims = np.array(ims).astype(np.uint8)
        if (ims.shape[-1] != 3):
            ims = ims.transpose((0,2,3,1))
    f = plt.figure(figsize=figsize)
    cols = len(ims)//rows if len(ims) % 2 == 0 else len(ims)//rows + 1
    for i in range(len(ims)):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i], interpolation=None if interp else 'none')
        
plots(imgs, titles=None) # titles=labels will display the image labels

In [ ]:
# End of Data Preparation
### ===================================================================================== ###
# Start of Model Building


### Set Up the Generators

In [ ]:
train_path = 'base_dir/train_dir'
valid_path = 'base_dir/val_dir'

num_train_samples = len(df_train)
num_val_samples = len(df_val)
train_batch_size = 10
val_batch_size = 10
image_size = 224

train_steps = np.ceil(num_train_samples / train_batch_size)
val_steps = np.ceil(num_val_samples / val_batch_size)

In [ ]:
train_batches = ImageDataGenerator(
    preprocessing_function= \
    keras.applications.mobilenet.preprocess_input).flow_from_directory(
                                                    train_path,
                                                    target_size=(image_size,image_size),
                                                    batch_size=train_batch_size,
                                                    class_mode='categorical')
valid_batches = ImageDataGenerator(
    preprocessing_function= \
    keras.applications.mobilenet.preprocess_input).flow_from_directory(
                                                    valid_path,
                                                    target_size=(image_size,image_size),
                                                    batch_size=val_batch_size,
                                                    class_mode='categorical')

# Note: shuffle=False causes the test dataset to not be shuffled
test_batches = ImageDataGenerator(
    preprocessing_function= \
    keras.applications.mobilenet.preprocess_input).flow_from_directory(
                                                    valid_path,
                                                    target_size=(image_size,image_size),
                                                    batch_size=val_batch_size,
                                                    class_mode='categorical',
                                                    shuffle=False)

### Modify MobileNet Model

In [ ]:
# create a copy of a mobilenet model

mobile = keras.applications.mobilenet.MobileNet()

In [ ]:
mobile.summary()

In [ ]:
type(mobile.layers)

In [ ]:
# How many layers does MobileNet have?
len(mobile.layers)

In [ ]:
# CREATE THE MODEL ARCHITECTURE

# Exclude the last 5 layers of the above model.
# This will include all layers up to and including global_average_pooling2d_1
x = mobile.layers[-6].output

# Create a new dense layer for predictions
# 3 corresponds to the number of classes
x = Dropout(0.25)(x)
predictions = Dense(3, activation='softmax')(x)

# inputs=mobile.input selects the input layer, outputs=predictions refers to the
# dense layer we created above.

model = Model(inputs=mobile.input, outputs=predictions)

In [ ]:
model.summary()

In [ ]:
# We need to choose how many layers we actually want to be trained.

# Here we are freezing the weights of all layers except the
# last 40 layers in the new model.
# The last 40 layers of the model will be trained.

for layer in model.layers[:-40]:
    layer.trainable = False

### Train the Model¶

In [ ]:
model.compile(Adam(lr=0.001), loss='categorical_crossentropy', 
              metrics=[categorical_accuracy])

In [ ]:
# Get the labels that are associated with each index
print(valid_batches.class_indices)

In [ ]:
# Add weights to try to make the model more sensitive to a specific class

class_weights={
    0: 1.0, # bacterial
    1: 1.0, # normal
    2: 1.0, # viral
}

In [ ]:
filepath = "model.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_categorical_accuracy', verbose=1, 
                             save_best_only=True, mode='max')

reduce_lr = ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=0.5, patience=2, 
                                   verbose=1, mode='max', min_lr=0.00001)
                              
                              
callbacks_list = [checkpoint, reduce_lr]

history = model.fit_generator(train_batches, steps_per_epoch=train_steps, 
                              #class_weight=class_weights,
                    validation_data=valid_batches,
                    validation_steps=val_steps,
                    epochs=30, verbose=1,
                   callbacks=callbacks_list)

### Evaluate the model using the val set¶

In [ ]:
# get the metric names so we can use evaulate_generator
model.metrics_names

In [ ]:
# Here the the last epoch will be used.

val_loss, val_cat_acc = \
model.evaluate_generator(test_batches, 
                        steps=val_steps)

print('val_loss:', val_loss)
print('val_cat_acc:', val_cat_acc)

In [ ]:
# Here the best epoch will be used.

model.load_weights('model.h5')

val_loss, val_cat_acc = \
model.evaluate_generator(test_batches, 
                        steps=val_steps)

print('val_loss:', val_loss)
print('val_cat_acc:', val_cat_acc)

### Plot the Training Curves

In [ ]:
# display the loss and accuracy curves

import matplotlib.pyplot as plt

acc = history.history['categorical_accuracy']
val_acc = history.history['val_categorical_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.figure()

plt.plot(epochs, acc, 'bo', label='Training cat acc')
plt.plot(epochs, val_acc, 'b', label='Validation cat acc')
plt.title('Training and validation cat accuracy')
plt.legend()
plt.figure()

### Create a Confusion Matrix

In [ ]:
# Get the labels of the test images.
# Note that cats and dogs are in seperate folders therefore
# the code below can get the labels depending on the folder the image is in.

test_labels = test_batches.classes

In [ ]:
# We need these to plot the confusion matrix.
test_labels

In [ ]:
# Print the label associated with each class
test_batches.class_indices

In [ ]:
# make a prediction
predictions = model.predict_generator(test_batches, steps=val_steps, verbose=1)

In [ ]:
predictions.shape

In [ ]:
# Source: Scikit Learn website
# http://scikit-learn.org/stable/auto_examples/
# model_selection/plot_confusion_matrix.html#sphx-glr-auto-examples-model-
# selection-plot-confusion-matrix-py


def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [ ]:
test_labels.shape

In [ ]:
# argmax returns the index of the max value in a row
cm = confusion_matrix(test_labels, predictions.argmax(axis=1))

In [ ]:
test_batches.class_indices

In [ ]:
# Define the labels of the class indices. These need to match the 
# order shown above.
cm_plot_labels = ['bacterial', 'normal', 'viral']

plot_confusion_matrix(cm, cm_plot_labels, title='Confusion Matrix')

### Create a Classification Report

In [ ]:
# Get the filenames, labels and associated predictions

# This outputs the sequence in which the generator processed the test images
test_filenames = test_batches.filenames

# Get the true labels
y_true = test_batches.classes

# Get the predicted labels
y_pred = predictions.argmax(axis=1)

In [ ]:
from sklearn.metrics import classification_report

# Generate a classification report


report = classification_report(y_true, y_pred, target_names=cm_plot_labels)

print(report)

### Display Some Incorrect Predictions

There seems to be many instances where the model found it hard to diffrentiate between bacterial pneumonia images and viral pneumonia images. Is this because these images are similar in some way? Lets print some images and take a look.

In [ ]:
# Get the filenames, labels and associated predictions

test_filenames = test_batches.filenames
test_labels = test_batches.classes
preds = predictions.argmax(axis=1)



In [ ]:
# check the lengths of these lists
print(len(test_filenames))
print(len(test_labels))
print(len(preds))

In [ ]:
# Put the above into a dataframe
pred_dict = {'filenames': test_filenames, 'labels': test_labels, 'predictions': preds}
df_preds = pd.DataFrame(pred_dict)

df_preds.head()

In [ ]:
# get the indices for the labels
test_batches.class_indices

In [ ]:
# filter out rows where the label was bacterial (0) and the model predicted viral (2)
df_1 = df_preds[(df_preds['labels'] == 0) & (df_preds['predictions'] == 2)]

# reset the index
df_1.reset_index(inplace=True, drop=True)

df_1.head()

In [ ]:
img_0 = val_dir + '/' + df_1.loc[0, 'filenames']
img_1 = val_dir + '/' + df_1.loc[1, 'filenames']
img_2 = val_dir + '/' + df_1.loc[2, 'filenames']
img_3 = val_dir + '/' + df_1.loc[3, 'filenames']

In [ ]:
# These are 4 bacterial pneumonia images that the model mis-classified as viral pneumonia.

plt.figure(figsize=(10,10))

plt.subplot(2,2,1)
plt.imshow(plt.imread(img_0), cmap='gray')
plt.axis('off')

plt.subplot(2,2,2)
plt.imshow(plt.imread(img_1), cmap='gray')
plt.axis('off')

plt.subplot(2,2,3)
plt.imshow(plt.imread(img_2), cmap='gray')
plt.axis('off')

plt.subplot(2,2,4)
plt.imshow(plt.imread(img_3), cmap='gray')
plt.axis('off')

In [ ]:
# Now let's print some true viral pneumonia images

df_2 = df_preds[(df_preds['labels'] == 2) & (df_preds['predictions'] == 2)]

# reset the index
df_2.reset_index(inplace=True, drop=True)

df_2.head()

In [ ]:
img_0 = val_dir + '/' + df_2.loc[0, 'filenames']
img_1 = val_dir + '/' + df_2.loc[1, 'filenames']
img_2 = val_dir + '/' + df_2.loc[2, 'filenames']
img_3 = val_dir + '/' + df_2.loc[3, 'filenames']

plt.figure(figsize=(10,10))


plt.subplot(2,2,1)
plt.imshow(plt.imread(img_0), cmap='gray')
plt.axis('off')

plt.subplot(2,2,2)
plt.imshow(plt.imread(img_1), cmap='gray')
plt.axis('off')

plt.subplot(2,2,3)
plt.imshow(plt.imread(img_2), cmap='gray')
plt.axis('off')

plt.subplot(2,2,4)
plt.imshow(plt.imread(img_3), cmap='gray')
plt.axis('off')

I think this is where domain knowledge would be very helpful. A radiologist would be able to look at these two sets of images and tell us what is the similarity between them that the model is detecting. 

*In bacterial pneumonia, there will likely be a much more visible presence of fluid in the lungs than viral pneumonia.* Is the model predicting bacterial instead of viral because it's seeing signs of fluid or is something else confusing it? It's a mystery.

In [ ]:
# End of Model Building
### ===================================================================================== ###
# Convert the Model from Keras to Tensorflow.js


### Install Tensorflow.js

In [ ]:
!pip install tensorflowjs

### Convert the model
Note: Do not load a saved model and try to convert it. It will not work.

In [ ]:
# create a directory to store the model files
os.mkdir('tfjs_dir')

# convert to Tensorflow.js
import tensorflowjs as tfjs

tfjs.converters.save_keras_model(model, 'tfjs_dir')

In [ ]:
# check the the directory containing the model is available
!ls

In [ ]:
# view the files that make up the tensorflow.js model
os.listdir('tfjs_dir')

In [ ]:
# Delete the image data directory we created to prevent a Kaggle error.
# Kaggle allows a max of 500 files to be saved.

shutil.rmtree('base_dir')

### Resources

Excellent tutorial series by deeplizard on how to use Mobilenet with Tensorflow.js<br>
https://www.youtube.com/watch?v=HEQDRWMK6yY

Tutorial by Minsuk Heo on Accuracy, Precision and F1 Score<br>
https://www.youtube.com/watch?v=HBi-P5j0Kec

Tutorial by Data School on how to evaluate a classifier<br>
https://www.youtube.com/watch?v=85dtiMz9tSo

Tensorflow.js gallery of projects<br>
https://github.com/tensorflow/tfjs/blob/master/GALLERY.md



Many thanks to Paul Mooney for posting this dataset. 

Thank you for reading.